<a href="https://colab.research.google.com/github/dhanushnayak/BERT/blob/main/BERT_SPAM_DETECTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 34.0 MB/s 
     |████████████████████████████████| 101 kB 4.2 MB/s 
     |████████████████████████████████| 596 kB 50.7 MB/s 
     |████████████████████████████████| 6.6 MB 28.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("/content/spam.csv",encoding="ISO-8859-1").iloc[:,:2]

In [21]:
from sklearn.model_selection import train_test_split

x = df.iloc[:,1].to_list()
y = df.iloc[:,0].map({"spam":1,"ham":0}).to_list()

In [22]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3)

In [31]:
from transformers import BertTokenizer,BertForSequenceClassification,TrainingArguments,Trainer

In [24]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased",do_lower_case=True)

In [25]:
train_encoding = tokenizer(x_train,padding=True)
test_encoding = tokenizer(x_test,padding=True)

In [26]:
train_encoding.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [38]:
import torch
from torch.utils.data import TensorDataset,Dataset
class CreateDataset(Dataset):
  def __init__(self,encoding,labels):
    self.encoding = encoding
    self.labels = labels
  def __getitem__(self,idx):
    item = {key: torch.tensor(val[idx]) for key,val in self.encoding.items()}
    item['labels']=torch.tensor(self.labels[idx])
    return item
  def __len__(self):
    return len(self.labels)

trainset = CreateDataset(train_encoding,y_train)

testset = CreateDataset(test_encoding,y_test)

In [30]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [41]:
args = TrainingArguments(
  output_dir="/content/outmodel", 
  overwrite_output_dir=True,
  logging_dir='/logs',
  num_train_epochs=1,
  logging_steps=10,
  per_device_train_batch_size=16,
  per_device_eval_batch_size=64,
  weight_decay=0.1,
  learning_rate=1e-5
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [42]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=trainset,
    eval_dataset= testset
)

In [43]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3900
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 244


Step,Training Loss
10,0.050900
20,0.042800
30,0.024800
40,0.021900
50,0.013900
60,0.003200
70,0.024900
80,0.002600
90,0.002900
100,0.001700




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=244, training_loss=0.025517721967695313, metrics={'train_runtime': 157.6202, 'train_samples_per_second': 24.743, 'train_steps_per_second': 1.548, 'total_flos': 442920739482000.0, 'train_loss': 0.025517721967695313, 'epoch': 1.0})

In [49]:
trainer.save_model("/content/modelr")

Saving model checkpoint to /content/modelr
Configuration saved in /content/modelr/config.json
Model weights saved in /content/modelr/pytorch_model.bin


In [44]:
from transformers import pipeline

In [50]:
model_r = pipeline("text-classification",model="/content/modelr",tokenizer=tokenizer)

loading configuration file /content/modelr/config.json
Model config BertConfig {
  "_name_or_path": "/content/modelr",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading configuration file /content/modelr/config.json
Model config BertConfig {
  "_name_or_path": "/content/modelr",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_p

In [57]:
model_r(x_test[9])

[{'label': 'LABEL_1', 'score': 0.9980745315551758}]

In [59]:
y_test[9],x_test[9]


(1,
 'REMINDER FROM O2: To get 2.50 pounds free call credit and details of great offers pls reply 2 this text with your valid name, house no and postcode')